In [1]:
import pandas as pd
import json
import ast
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

# Lectura de un archivo JSON y Creación de un DataFrame

Este código abre y lee un archivo JSON llamado 'users_items.json', convirtiendo sus datos en un DataFrame de Pandas. Utiliza la biblioteca `pandas` y la función `ast.literal_eval()` para procesar cada línea del archivo y crear el DataFrame.

El proceso puede ser demorado, dependiendo del tamaño del archivo.



In [ ]:
row = []
with open('./users_items.json', encoding='latin1') as f:
    for lines in f.readlines():
        row.append(ast.literal_eval(lines)) 
df = pd.DataFrame(row) 

El código `df.head()` muestra las primeras 5 filas del DataFrame `df`, proporcionando una vista previa de los datos.


In [ ]:
df.head()

In [ ]:
rows = []
with open ('./user_reviews.json', encoding='latin1') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df1 = pd.DataFrame(rows) #Demora 1.6s

In [ ]:

df1.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
data = []

with open('./steam_games.json') as f:
    for line in f:
        try:
            json_data = json.loads(line)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {line}")

df_output = pd.DataFrame(data) #Demora 1.2s

In [ ]:
df_output.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None


El código `df.info()` proporciona información resumida sobre el DataFrame `df`, incluyendo el tipo de datos de cada columna y la cantidad de valores no nulos, lo que es útil para comprender la estructura y calidad de los datos.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


# Desanidación de Datos Anidados

Este código tiene como objetivo principal desanidar datos anidados contenidos en el DataFrame original (`df`) y crear un nuevo DataFrame llamado `df_desanidado`. Los datos anidados se encuentran en una columna llamada 'items'.

## Pasos Explicados
1. **Iteración a través del DataFrame Original**: Se utiliza un bucle `for` para recorrer cada fila del DataFrame original (`df`). En cada iteración, se extraen los valores relevantes, como 'user_id', 'items_count', 'steam_id', 'user_url', y 'items'.

2. **Desanidación de Datos**: Se ingresa en un segundo bucle `for` para iterar a través de los elementos en la columna 'items' de cada fila. Para cada elemento en 'items', se extraen valores como 'item_id', 'item_name', 'playtime_forever', y 'playtime_2weeks'.

3. **Creación de Nuevas Filas**: Se crea un nuevo diccionario (`new_row`) que contiene todos estos valores extraídos, así como los valores previamente extraídos de la fila original. Esto se hace para mantener la asociación entre los datos de los juegos y los datos del usuario.

4. **Agregación de Filas Desanidadas**: Cada `new_row` se agrega a una lista llamada `data_desanidada`, que acumula todas las filas desanidadas.

5. **Creación del DataFrame Desanidado**: Finalmente, se crea un nuevo DataFrame (`df_desanidado`) a partir de la lista `data_desanidada`, que contiene todos los datos desanidados.

## Tiempo de Ejecución
Este proceso puede ser demorado, aproximadamente 13.7 segundos en este caso, debido a la iteración a través de las filas y elementos anidados. Sin embargo, el resultado es un DataFrame `df_desanidado` que contiene los datos en un formato más estructurado y adecuado para análisis posteriores.


In [ ]:

data_desanidada = []

for index, row in df.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        data_desanidada.append(new_row)

df_desanidado = pd.DataFrame(data_desanidada) #Demora 13.7s

# Desanidación de Datos con Replicación

En este caso, al desanidar la columna 'items', se replicaron datos en otras columnas existentes para evitar pérdida de información. Esto significa que algunos campos pueden contener valores duplicados debido al proceso de desanidación.

**Motivo de la Replicación:**
La replicación de datos se realizó para garantizar que la información relacionada con cada juego (como 'item_id', 'item_name', 'playtime_forever', y 'playtime_2weeks') permanezca asociada con los datos del usuario ('user_id', 'items_count', 'steam_id', 'user_url').

**Uso de `dropna()`:**
El uso de `dropna()` no sería apropiado en este caso, ya que podría eliminar filas que contienen datos duplicados necesarios para mantener la relación entre los juegos y los usuarios. En lugar de eliminar filas con valores nulos, se optó por replicar datos para preservar la integridad de la información.

Esta estrategia asegura que los datos desanidados conserven su estructura y que los análisis posteriores se realicen de manera adecuada sin pérdida de información relevante.


In [ ]:
df_desanidado.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


# Selección de Columnas de Interés

En este paso, se eligen las columnas específicas que son relevantes para la API y el modelamiento. Las columnas seleccionadas incluyen información esencial como 'user_id', 'user_url', 'items_count', 'item_id', 'item_name', y 'playtime_forever'. Estas columnas son fundamentales para los análisis y procesos posteriores.

A continuación, se muestran las primeras filas de estas columnas seleccionadas con `df_columns.head()`.


In [ ]:
df_columns = df_desanidado[['user_id', 'user_url', 'items_count', 'item_id', 'item_name', 'playtime_forever']]
df_columns.head()

,user_id,user_url,items_count,item_id,item_name,playtime_forever
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,10,Counter-Strike,6
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,20,Team Fortress Classic,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,30,Day of Defeat,7
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,40,Deathmatch Classic,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,50,Half-Life: Opposing Force,0


# II. Siguiendo con los datos de 'Reviews'

In [ ]:
df1.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
df1.info() #Vemos 1 null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


# Desanidación de Datos de Reseñas

En este código, se desanidan datos anidados contenidos en el DataFrame `df1`. La columna 'reviews' se desanida para crear un nuevo DataFrame llamado `df1_desanidado`.

Los datos desanidados incluyen información como 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', y 'sentiment_analysis'. Estos datos son relevantes para el análisis de reseñas de usuarios.

El proceso es rápido, aproximadamente 1.0 segundo, y el resultado es un DataFrame estructurado para análisis de reseñas.


In [ ]:
df1_desanidada = []

for index, row in df1.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    
    for i in reviews:   
        new_row = {
        'user_id': user_id,
        'user_url' : user_url,
        'funny': i.get('funny', ''),
        'posted': i.get('posted', ''),
        'last_edited': i.get('last_edited', ''),
        'item_id': i.get('item_id', ''),
        'helpful': i.get('helpful', ''),
        'recommend': i.get('recommend', ''),
        'sentiment_analysis': i.get('review', ''),         
        }
    df1_desanidada.append(new_row)

df1_desanidado = pd.DataFrame(df1_desanidada) #Demora 1.0s

# Limpieza de Datos

En este código, se realiza una limpieza de datos en el DataFrame `df1_desanidado`. Se eliminan las filas que contienen valores NaN en la columna 'sentiment_analysis' utilizando `df1_desanidado['sentiment_analysis'].notna()`.

Además, se rellenan los valores NaN restantes en la columna 'sentiment_analysis' con una cadena vacía utilizando `df1_desanidado['sentiment_analysis'].fillna('', inplace=True)`.

Este proceso garantiza que la columna 'sentiment_analysis' esté libre de valores NaN y que los valores nulos se reemplacen por cadenas vacías, lo que facilita su procesamiento y análisis posterior.


In [ ]:
df1_desanidado = df1_desanidado[df1_desanidado['sentiment_analysis'].notna()]

df1_desanidado['sentiment_analysis'].fillna('', inplace=True)

In [ ]:
df1_desanidado.head()
#En este caso vemos que hay columnas que no tienen datos a simple vista, como funny o last_edited

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,224500,No ratings yet,True,"Fun world builder, with plenty of option of ho..."
3,doctr,http://steamcommunity.com/id/doctr,,"Posted February 23, 2012.",,108710,No ratings yet,True,"Alan wake is a really good game, the light eff..."
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted July 11, 2013.",,204300,No ratings yet,True,"OH YES, THIS GAME IS THE BEST, THEY ADD STUFF ..."


In [ ]:
df1_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25799 non-null  object
 1   user_url            25799 non-null  object
 2   funny               25799 non-null  object
 3   posted              25799 non-null  object
 4   last_edited         25799 non-null  object
 5   item_id             25799 non-null  object
 6   helpful             25799 non-null  object
 7   recommend           25799 non-null  bool  
 8   sentiment_analysis  25799 non-null  object
dtypes: bool(1), object(8)
memory usage: 1.6+ MB


# Análisis de Sentimientos con VADER

En este código, se utiliza la biblioteca VADER (Valence Aware Dictionary and sEntiment Reasoner) para realizar un análisis de sentimientos en las reseñas contenidas en la columna 'sentiment_analysis' del DataFrame `df1_desanidado`.

## Pasos Explicados
1. **Inicialización de VADER**: Se inicializa VADER mediante la creación de un objeto `SentimentIntensityAnalyzer` y se proporciona el archivo de léxico VADER como `lexicon_file='./vader_lexicon.txt'`.

2. **Agregación de Métricas**: Se calculan cuatro métricas de sentimiento diferentes para cada reseña utilizando VADER: 'compound', 'neg' (negativo), 'neu' (neutral) y 'pos' (positivo). Estas métricas se agregan al DataFrame `df1_desanidado` mediante el uso de `apply()` y las funciones lambda.

Este proceso permite medir y agregar información sobre el sentimiento de cada reseña en el DataFrame, lo que es útil para análisis posteriores relacionados con el sentimiento de las reseñas.


In [ ]:
# Inicializamos VADER
sia = SentimentIntensityAnalyzer(lexicon_file='./vader_lexicon.txt')

# Agregamos VADER a las metricas del Dataframe
df1_desanidado['compound'] = df1_desanidado['sentiment_analysis'].apply(lambda x: sia.polarity_scores(x)['compound'])
df1_desanidado['neg'] = df1_desanidado['sentiment_analysis'].apply(lambda x: sia.polarity_scores(x)['neg'])
df1_desanidado['neu'] = df1_desanidado['sentiment_analysis'].apply(lambda x: sia.polarity_scores(x)['neu'])
df1_desanidado['pos'] = df1_desanidado['sentiment_analysis'].apply(lambda x: sia.polarity_scores(x)['pos'])

In [ ]:
df1_copy = df1_desanidado.copy()
#Función para categorizar el sentimiento según la puntuación compuesta
def categorize_sentiment(compound):
    if compound >= 0.05:
        return 2
    elif compound <= -0.05:
        return 0
    else:
        return 1

df1_copy['sentiment_analysis'] = df1_copy['compound'].apply(categorize_sentiment)

df1_copy

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,compound,neg,neu,pos
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,2,0.9117,0.000,0.704,0.296
1,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,2,0.7713,0.081,0.541,0.378
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,224500,No ratings yet,True,2,0.5106,0.093,0.643,0.264
3,doctr,http://steamcommunity.com/id/doctr,,"Posted February 23, 2012.",,108710,No ratings yet,True,2,0.9571,0.000,0.738,0.262
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted July 11, 2013.",,204300,No ratings yet,True,2,0.9939,0.071,0.313,0.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,,Posted May 31.,,261030,0 of 1 people (0%) found this review helpful,True,2,0.8015,0.161,0.493,0.345
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,,Posted June 17.,,730,0 of 1 people (0%) found this review helpful,True,1,0.0000,0.000,1.000,0.000
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,1 person found this review funny,Posted June 23.,,570,1 of 1 people (100%) found this review helpful,True,2,0.2732,0.000,0.323,0.677
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,2,0.9786,0.026,0.692,0.282


# Eliminación de Columnas

En este código, se eliminan las columnas 'compound', 'neg', 'neu' y 'pos' del DataFrame `df1_copy` utilizando `df1_copy.drop(['compound', 'neg', 'neu', 'pos'], axis=1)`.

La eliminación de estas columnas puede ser necesaria cuando se desea reducir el tamaño del DataFrame o cuando estas columnas ya no son relevantes para el análisis posterior.

El resultado es un DataFrame `df1_copy` que ahora carece de estas columnas específicas.


In [ ]:
df1_copy = df1_copy.drop(['compound', 'neg', 'neu', 'pos'], axis=1)
df1_copy

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,224500,No ratings yet,True,2
3,doctr,http://steamcommunity.com/id/doctr,,"Posted February 23, 2012.",,108710,No ratings yet,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted July 11, 2013.",,204300,No ratings yet,True,2
...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,,Posted May 31.,,261030,0 of 1 people (0%) found this review helpful,True,2
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,,Posted June 17.,,730,0 of 1 people (0%) found this review helpful,True,1
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,1 person found this review funny,Posted June 23.,,570,1 of 1 people (100%) found this review helpful,True,2
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,2


# Limpieza y Formateo de Fechas

En este código, se realiza una serie de pasos para crear y formatear la columna 'clean_date' en el DataFrame `df1_copy`.

## Paso 1: Creación de 'clean_date'
Se crea la columna 'clean_date' utilizando la columna 'posted' y aplicando una función que elimina la palabra 'Posted' y los puntos en cada entrada utilizando expresiones regulares y `re.sub()`. Además, se utiliza `strip()` para eliminar cualquier espacio en blanco al principio o al final.

## Paso 2: Conversión a Formato de Fecha
La columna 'clean_date' se convierte en formato de fecha utilizando `pd.to_datetime()`. Los valores que no se pueden convertir en fechas válidas se manejan con 'errors='coerce''.

## Paso 3: Formateo de Fecha
La columna 'clean_date' se vuelve a formatear en una cadena con el formato 'yyyy-mm-dd' utilizando `dt.strftime()`.

El resultado es el DataFrame `countreviews` que contiene la información de reseñas junto con la nueva columna 'clean_date' formateada como fecha.


In [ ]:
df1_copy['clean_date'] = df1_copy['posted'].apply(lambda x: re.sub(r'Posted|\.', '', x).strip())
countreviews = df1_copy[['user_id','item_id', 'recommend', 'sentiment_analysis', 'clean_date']].copy()

countreviews['clean_date'] = pd.to_datetime(df1_copy['clean_date'], errors='coerce')

countreviews['clean_date'] = countreviews['clean_date'].dt.strftime('%Y-%m-%d')

# Selección de Columnas de Interés

En este código, se seleccionan específicamente las columnas de interés del DataFrame `countreviews`. Las columnas seleccionadas incluyen 'user_id', 'item_id', 'recommend', 'sentiment_analysis' y 'clean_date'.

Se utiliza `dropna()` para eliminar filas que contienen valores nulos en estas columnas, garantizando que los datos sean completos y adecuados para su uso.

El resultado es el DataFrame `df1_columns`, que contiene las columnas seleccionadas y limpias.

A continuación, se muestra una vista previa de las primeras filas de este subconjunto de datos con `df1_columns.head()`.


In [ ]:
df1_columns = countreviews[['user_id','item_id', 'recommend', 'sentiment_analysis','clean_date']].dropna()
df1_columns.head()

,user_id,item_id,recommend,sentiment_analysis,clean_date
0,76561197970982479,43110,True,2,2011-04-21
1,js41637,239030,True,2,2013-11-29
2,evcentric,224500,True,2,2014-10-15
3,doctr,108710,True,2,2012-02-23
4,maplemage,204300,True,2,2013-07-11


# III. Seguimos con el ultimo dataset de 'Games'

In [ ]:
df_output.info() #Se ven muchos null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [ ]:
df2 = df_output.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22530 non-null  object
 1   genres        22530 non-null  object
 2   app_name      22530 non-null  object
 3   title         22530 non-null  object
 4   url           22530 non-null  object
 5   release_date  22530 non-null  object
 6   tags          22530 non-null  object
 7   reviews_url   22530 non-null  object
 8   specs         22530 non-null  object
 9   price         22530 non-null  object
 10  early_access  22530 non-null  object
 11  id            22530 non-null  object
 12  developer     22530 non-null  object
dtypes: object(13)
memory usage: 2.4+ MB


In [ ]:
df2

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


# Renombrar Columna

En este código, se lleva a cabo una operación de renombrado de columna en el DataFrame `df2`. La columna 'id' se renombra como 'item_id' utilizando la función `rename()`.

Este cambio de nombre es útil cuando se desea que la columna 'id' sea más descriptiva y representativa, en este caso, como 'item_id'. El DataFrame resultante se almacena en `df2_output`.

El renombrado de columnas es una práctica común para mejorar la claridad y la comprensión de los datos.


In [ ]:
df2_output = df2.rename(columns={'id': 'item_id'}) 

# Selección de Columnas de Interés

En este código, se seleccionan específicamente las columnas de interés del DataFrame `df2_output`. Las columnas seleccionadas incluyen 'genres', 'release_date', 'price', 'item_id' y 'developer'.

El DataFrame resultante se almacena en `df2_columns`, y contiene solo estas columnas seleccionadas, lo que puede ser útil para análisis y tareas específicas.

A continuación, se muestra el contenido de `df2_columns`.


In [ ]:
df2_columns = df2_output[['genres', 'release_date','price', 'item_id', 'developer']]
df2_columns

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,Free To Play,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,Free to Play,670290,Poolians.com
88313,"[Action, Adventure, Casual]",2017-12-07,0.99,767400,彼岸领域
88315,"[Action, Adventure, Simulation]",2018-01-04,3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...
120439,"[Action, Adventure, Casual, Indie]",2018-01-04,1.99,745400,Bidoniera Games
120440,"[Casual, Indie, Simulation, Strategy]",2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",2018-01-04,4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",2018-01-04,1.99,610660,Laush Dmitriy Sergeevich


# IV. Creamos nuestros csv con nuestros datos limpos

# Creación de Archivo CSV

En este código, se toma un subconjunto de datos del DataFrame `df_columns` utilizando `sample(n=40000)`. Este subconjunto incluye 40,000 filas de datos seleccionadas aleatoriamente.

Luego, el subconjunto de datos se guarda como un nuevo archivo CSV en la ruta especificada (`csv_file_path`) utilizando `to_csv()`. El parámetro `index=False` se utiliza para evitar la inclusión del índice en el archivo CSV resultante.

El proceso de creación del archivo CSV demora aproximadamente 0.2 segundos y genera un archivo que contiene el subconjunto de datos seleccionado.


In [ ]:
csv_file_path = './new_users_item.csv'
df = df_columns.sample(n=40000)
df.to_csv(csv_file_path, index=False) #Demora 0.2s

# Creación de Archivo CSV

En este código, se toma el DataFrame `df1_columns`, que contiene las columnas seleccionadas, y se guarda como un nuevo archivo CSV en la ruta especificada (`csv_file_path`) utilizando `to_csv()`. 

El proceso demora aproximadamente 0.2 segundos y genera un archivo CSV con los datos de interés.


In [ ]:
csv_file_path = './new_users_reviews.csv'

df1_columns.to_csv(csv_file_path, index=False) #Demora 0.2s

# Creación de Archivo CSV

En este último paso del proceso ETL (Extract, Transform, Load), se toma el DataFrame `df2_columns`, que contiene las columnas de interés relacionadas con juegos de Steam. Luego, se guarda el contenido de este DataFrame como un nuevo archivo CSV en la ruta especificada (`csv_file_path`) utilizando `to_csv()`.

El parámetro `index=False` se utiliza para evitar la inclusión del índice en el archivo CSV resultante.

Este proceso demora aproximadamente 0.2 segundos y genera un archivo CSV que contiene los datos finales relacionados con los juegos de Steam.

Así, concluye el proceso ETL, habiendo extraído, transformado y cargado los datos necesarios para su posterior análisis y uso.


In [ ]:
csv_file_path = './new_steam_games.csv'
df2_columns.to_csv(csv_file_path, index=False) #Demora 0.2s